# Simplex
## March 3rd, 2022
### Overview: Using the simplex method to optimize linear constraint problems

In [1]:
import numpy as np

In [2]:
# Problems 1-6
class SimplexSolver(object):
    """Class for solving the standard linear optimization problem

                        minimize        c^Tx
                        subject to      Ax <= b
                                         x >= 0
    via the Simplex algorithm.
    """
    # Problem 1
    def __init__(self, c, A, b):
        """Check for feasibility and initialize the dictionary.

        Parameters:
            c ((n,) ndarray): The coefficients of the objective function.
            A ((m,n) ndarray): The constraint coefficients matrix.
            b ((m,) ndarray): The constraint vector.

        Raises:
            ValueError: if the given system is infeasible at the origin.
        """
        origin = np.zeros(len(A[0]))
        if not all(np.greater(b,A@origin)): raise ValueError("infeasible at origin")
        self.c = c
        self.A = A
        self.b = b
        self._generatedictionary(c,A,b)

    # Problem 2
    def _generatedictionary(self, c, A, b):
        """Generate the initial dictionary.

        Parameters:
            c ((n,) ndarray): The coefficients of the objective function.
            A ((m,n) ndarray): The constraint coefficients matrix.
            b ((m,) ndarray): The constraint vector.
        """
        #slack variables
        w = np.ones(len(A))
        
        #create all the bar vectors, matrices
        cbar = np.concatenate((c,np.zeros(len(w))))
        Abar = np.hstack((A,np.identity(len(w))))
        bbar = np.concatenate((np.array([0]),b))
        bbar = np.reshape(bbar,(len(bbar),1))
        right = np.vstack((cbar,-Abar))
        
        #dictionary
        self.D = np.hstack((bbar,right))

    # Problem 3a
    def _pivot_col(self):
        """Return the column index of the next pivot column."""
        #find the first negative value in the objective function coefficients
        for ind in range(1,len(self.D[0])):
            if self.D[0][ind] < 0:
                break
                
        #save that column index
        self.colInd = ind
        return ind

    # Problem 3b
    def _pivot_row(self,index=0):
        """Determine the row index of the next pivot row using the ratio test
        (Bland's Rule).
        """
        #calculate the pivot column
        self._pivot_col()
        
        #get the numerators and the denominators for the ratios
        numers = -np.array([self.D.T[0][i] for i in range(1,len(self.D))])
        denoms = np.array([self.D.T[self.colInd][i] for i in range(1,len(self.D))])+1e-16
        ratios = numers/denoms
        
        #disregard all negative ratios by setting to infinity
        for i, el in enumerate(ratios):
            if el < 0: ratios[i] = np.inf
                
        #the row index is the index of the smallest ratio
        self.rowInd = np.argmin(ratios)+1
        
        return self.rowInd

    # Problem 4
    def pivot(self):
        """Select the column and row to pivot on. Reduce the column to a
        negative elementary vector.
        """
        #do pivot row function to calc both column and row indices
        self._pivot_row()
        
        #checking if unbounded
        if True not in np.greater(np.zeros(len(self.D.T[0])),self.D.T[self.colInd]):
            raise ValueError("Problem is unbounded")
        
        #dividing pivot row by negative of pivot element
        self.D[self.rowInd] /= -self.D[self.rowInd][self.colInd]
        
        #identify modified pivot element
        piv_el = self.D[self.rowInd][self.colInd]
        
        #for each row (except pivot row)
        for i in range(len(self.D)):
            if i == self.rowInd:
                pass
            #do some silly row operations with the silly willy dictionary
            else:
                self.D[i] -= (self.D[i][self.colInd]/piv_el)*self.D[self.rowInd]
    
    # Problem 5
    def solve(self):
        """Solve the linear optimization problem.

        Returns:
            (float) The minimum value of the objective function.
            (dict): The basic variables and their values.
            (dict): The nonbasic variables and their values.
        """
        #run until can minimize no further
        while True in np.greater( np.zeros(len(self.D[0])-1),  self.D[0][1:]):
            self.pivot()
        
        #the minimum value is the first entry
        self.min = self.D[0][0]
        
        #find where first value that is nonzero, which is start of variables not in objective function (not mapped to zero)
        FirstnonZeroInd = (np.argwhere(self.D[0][1:]!=0).flatten()+1)[0]
        ZeroInds = np.argwhere(self.D[0][1:]==0).flatten()+1
        
        #init dicts
        self.Maps = dict()
        self.Maps2 = dict()

        
        for i in range(1,len(self.D)):
            if -1 in self.D[i][1:]:
                xind = np.argwhere(self.D[i][1:] == -1).flatten()[0]+1
                if xind in ZeroInds and xind < FirstnonZeroInd:
                    self.Maps[xind] = self.D[i][0]
        
        for ind in range(1,len(self.D[0])):
            if ind+1 not in self.Maps.keys():
                self.Maps2[ind+1] = 0
        
        
        return self.min, self.Maps, self.Maps2

In [3]:
c = np.array([-3,-2])
b = np.array([2,5,7])
A = np.array([
    [1,-1],
    [3,1],
    [4,3]
])

In [4]:
simp = SimplexSolver(c,A,b)
simp.solve()

(-5.2,
 {3: 0.5999999999999996, 1: 1.6, 2: 0.19999999999999982},
 {4: 0, 5: 0, 6: 0})

In [5]:
# Problem 6
def prob6(filename='productMix.npz'):
    """Solve the product mix problem for the data in 'productMix.npz'.

    Parameters:
        filename (str): the path to the data file.

    Returns:
        ((n,) ndarray): the number of units that should be produced for each product.
    """
    #load the file
    dicts = np.load(filename)
    A = dicts['A']                    #resource coefficients
    p = dicts['p']                    #unit prices
    m = dicts['m']                    #available resource units
    d = dicts['d']                    #demand constraints
    
    #negative profit is the c vector
    c = -p
    #A matrix is the A stacked over the identity
    AA = np.vstack((A,np.identity(len(A[0]))))
    #b vector is m and d concatenated
    b = np.concatenate((m,d))

    #create simplex solver, solve, and access first dictionary
    p = SimplexSolver(c,AA,b).solve()[1]
    #getting indices to sort the values array (to be made) according to key values
    inds = np.argsort(list(p.keys()))
    #creating array of the values and ordering them according to key 
    Prob = np.array(list(p.values()))[inds]

    return Prob

In [6]:
prob6()

array([10.        ,  6.19298246, 12.        ,  1.78947368])